In [1]:
import cv2 #opencv读取格式BGR
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

pie = cv2.imread('pie.png')

cv2.imshow('pie', pie)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 1-Sobel算子
- Gx, Gy:右 - 左， 下 - 上

In [42]:
def cv_show(name, img):
    cv2.imshow(name,img)
    #等待时间毫秒级，0表示任意键终止
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
sobelx = cv2.Sobel(pie, cv2.CV_64F, 1, 0, ksize = 3)

cv_show('sobelx', sobelx)

## 2-梯度计算方法
负数会被截断成0，所以要取绝对值（右-左）

In [13]:
sobelx = cv2.Sobel(pie, cv2.CV_64F, 1, 0, ksize = 3)
sobelx = cv2.convertScaleAbs(sobelx)

cv_show('sobelx', sobelx)

In [12]:
sobely = cv2.Sobel(pie, cv2.CV_64F, 0, 1, ksize = 3)
sobely = cv2.convertScaleAbs(sobely)

cv_show('sobelx', sobely)

分别计算x和y，再求和

In [18]:
sobelxy = cv2.addWeighted(sobelx, 0.5, sobely, 0.5, 0)
cv_show('so', sobelxy)

不建议直接计算

In [19]:
sobely = cv2.Sobel(pie, cv2.CV_64F, 1, 1, ksize = 3)
sobely = cv2.convertScaleAbs(sobely)

cv_show('sobelx', sobely)

In [32]:
pie = cv2.imread('lena.jpg', cv2.IMREAD_GRAYSCALE)

cv2.imshow('pie', pie)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
sobelx = cv2.Sobel(pie, cv2.CV_64F, 1, 0, ksize = 3)
sobelx = cv2.convertScaleAbs(sobelx)

cv_show('sobelx', sobelx)

In [36]:
sobely = cv2.Sobel(pie, cv2.CV_64F, 0, 1, ksize = 3)
sobely = cv2.convertScaleAbs(sobely)

cv_show('sobelx', sobely)

In [37]:
sobelxy = cv2.addWeighted(sobelx, 0.5, sobely, 0.5, 0)
cv_show('so', sobelxy)

In [34]:
sobely = cv2.Sobel(pie, cv2.CV_64F, 1, 1, ksize = 3)
sobely = cv2.convertScaleAbs(sobely)

cv_show('sobelx', sobely)

## 3-scharr与lapkacian算子
- scharr 更细致
- lapkacian 导数，效果一般和工具一起使用

In [40]:
#不同算子的差异
scharrx = cv2.Scharr(pie, cv2.CV_64F, 1, 0)
scharry = cv2.Scharr(pie, cv2.CV_64F, 0, 1)
scharrx = cv2.convertScaleAbs(scharrx)
scharry = cv2.convertScaleAbs(scharry)
scharrxy = cv2.addWeighted(scharrx, 0.5, scharry, 0.5, 0)

laplacian = cv2.Laplacian(pie, cv2.CV_64F)
laplacian = cv2.convertScaleAbs(laplacian)

res = np.hstack((sobelxy, scharrxy, laplacian))
cv_show('res', res)

## 1-Canny边缘检测流程
- 1）使用高斯滤波，以平滑图像，滤除噪声。
- 2）计算图像中每个像素点的梯度强度和方向。
- 3）应用非极大值抑制，以消除边缘检测带来的杂散响应。
- 4）应用双阈值检测来确定真实的和潜在的边缘。
- 5）通过抑制孤立的弱边缘最终完成边缘检测。

In [48]:
img = cv2.imread('lena.jpg', cv2.IMREAD_GRAYSCALE)

v1 = cv2.Canny(img, 180, 250)
v2 = cv2.Canny(img, 50, 100)

res = np.hstack((v1, v2))
cv_show('res', res)

In [49]:
img = cv2.imread('car.png', cv2.IMREAD_GRAYSCALE)

v1 = cv2.Canny(img, 180, 250)
v2 = cv2.Canny(img, 50, 100)

res = np.hstack((v1, v2))
cv_show('res', res)

# 图像金字塔
## 1-图像金字塔定义
###  高斯金字塔
####   向下采样法（缩小）
- 将Gi与高斯内核卷积
- 将所有偶数行和列去除
####   向上采样法（放大）
- 将图像在每个方向扩大为原来的两倍，新增的行和列以0填充
- 使用先前同样的内核（乘以4）与放大后的图像卷积，获得近似值
- 拉普拉斯金字塔
###  拉普拉斯金字塔
- Li = Gi - PyrUp(PyrDown(Gi))
- 低通滤波、缩小尺寸、放大尺寸、图像相减

## 2-金字塔制作方法

In [50]:
img = cv2.imread('AM.png')

cv_show('res', img)
print(img.shape)

(442, 340, 3)


In [55]:
# 上采样
up = cv2.pyrUp(img)
cv_show('up', up)
print(up.shape)

(884, 680, 3)


In [56]:
# 下采样
down = cv2.pyrDown(img)
cv_show('down', down)
print(down.shape)

(221, 170, 3)


In [58]:
# 上采样
up2 = cv2.pyrUp(up)
cv_show('up2', up2)
print(up2.shape)

(1768, 1360, 3)


In [60]:
up_down = cv2.pyrDown(up)
cv_show('up_down', np.hstack((img, up_down)))

In [61]:
cv_show('img-up_down', img-up_down)

In [63]:
# 拉普拉斯金字塔

down = cv2.pyrDown(img)
down_up = cv2.pyrUp(down)

l_1 = img - down_up
cv_show('l_1', l_1)

# 图像轮廓
## 1-轮廓检测算法

In [82]:
img = cv2.imread('contours.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
cv_show('t', thresh)

In [83]:
contours, hieraychy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

## 2-轮廓检测结果

In [90]:
# 传入绘制图像，轮廓 轮廓索引 颜色模式 线条厚度
# 注意需要copy要不原图会变。。
draw_img = img.copy() 
res = cv2.drawContours(draw_img, contours, 2, (0, 0, 255), 2) #原图像 轮廓 -1指全部 BGR 宽度
cv_show('res', res)

In [86]:
cv_show('t', img)

## 3-轮廓特征与相似
### 轮廓特征

In [91]:
cnt = contours[0]

In [92]:
# 面积
cv2.contourArea(cnt)

8500.5

In [93]:
#周长
cv2.arcLength(cnt, True)

437.9482651948929

### 轮廓近似

In [131]:
img= cv2.imread('contours2.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #灰度图
# 二值
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
contours, hieraychy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
cnt = contours[1]

draw_img1 = img.copy() 
res = cv2.drawContours(draw_img1, [cnt], -1, (0, 0, 255), 2) #原图像 轮廓 -1指全部 BGR 宽度
cv_show('res', res)

In [129]:
img1= cv2.imread('contours.png')
epsilon = 0.01*cv2.arcLength(cnt, True)
approx = cv2.approxPolyDP(cnt, epsilon, True)

draw_img1 = img1.copy() 
res1 = cv2.drawContours(draw_img1, [approx], -1, (0, 0, 255), 2) #原图像 轮廓 -1指全部 BGR 宽度
cv_show('res1', res1)

### 边界矩形

In [137]:
img = cv2.imread('contours.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #灰度图
contours, hieraychy = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
cnt = contours[2]

x,y,w,h = cv2.boundingRect(cnt)
img = cv2.rectangle(img, (x, y), (x + w, y +h),  (0, 255, 0), 2)

cv_show('res', img)

In [110]:
area = cv2.contourArea(cnt)

x,y,w,h = cv2.boundingRect(cnt)
rect_area = w * h
extent = float(area) / rect_area

print('轮廓面积与边界矩形比', extent)

轮廓面积与边界矩形比 0.7800798598378357


### 外接圆

In [138]:
(x, y), radius = cv2.minEnclosingCircle(cnt)
center = (int(x), int(y))
radius = int(radius)
img = cv2.circle(img, center, radius, (0, 255, 0), 2)
cv_show('res', img)